In [ ]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

# library(msigdbr)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

Here I perform enrichment analysis to find modules enriched for cell type markers. These modules will later be used to correlate to exon PSI to find cell type-specific exons.

In [3]:
# Get DE genes frome each cell type: pooled analysis
pooled_res <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [ ]:
pval_threshold <- .05/length(pooled_res)

ctype_genes <- lapply(pooled_res, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > 2)
    df[mask, 1]
})
names(ctype_genes) <- names(pooled_res) 

In [15]:
network_dir <- "mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules"

In [ ]:
# Traverse networks to get cell type enrichments for each module

networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
networks <- networks[lengths(lapply(networks, list.files)) > 0]

network_enrichments <- lapply(seq_along(networks), function(i) {
    
    kme_file <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
    kme <- fread(file.path(networks[i], kme_file), data.table=FALSE)
    mod_col <- grep("PosFDR", colnames(kme))
    mod_genes <- tapply(kme$Gene, kme[,mod_col], list)
    
    if (length(mod_genes) > 0) {
        all_genes <- kme$Gene

        # For each module calculate enrichment for top kME genes
        mod_enrichments <- lapply(mod_genes, function(mod) {
            lapply(unlist(lapply(ctype_genes, function(set) {
                fisher_test(set, mod, all=all_genes)
            })), c)
        })
        
        network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
        mod_enrichments_df <- reshape2::melt(mod_enrichments)
        colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
        
        me_file <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
        
        data.frame(
            Network=network_id,
            kME_path=file.path(networks[i], kme_file),
            ME_path=file.path(networks[i], me_file),
            mod_enrichments_df
        )
    }
    
})
enrichments_df <- do.call(rbind, network_enrichments)

In [18]:
enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

In [21]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

top_mods_df <- enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    slice_min(Qval) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [23]:
top_mods_df

Network,kME_path,ME_path,Pval,Cell_type,Module,Qval
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/kME_table_06-44-39.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/Module_eigengenes_06-44-39.csv,1.157541e-23,SMC,pink,8.894890e-19
Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151/kME_table_06-18-43.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151/Module_eigengenes_06-18-43.csv,5.414983e-21,Endo,blue,3.782760e-17
Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151/kME_table_06-18-43.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize3_merge_ME_0.9_20151/Module_eigengenes_06-18-43.csv,7.031158e-21,VLMC,magenta,4.502461e-17
Bicor-None_signum0.586_minSize6_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize6_merge_ME_0.9_20151/kME_table_06-37-30.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.586_minSize6_merge_ME_0.9_20151/Module_eigengenes_06-37-30.csv,8.146626e-20,L2_3_IT,grey60,2.721788e-16
Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/kME_table_06-44-39.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/Module_eigengenes_06-44-39.csv,8.103133e-20,L6_CT,darkgreen,2.721788e-16
Bicor-None_signum0.325_minSize8_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize8_merge_ME_0.9_20151/kME_table_07-19-33.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize8_merge_ME_0.9_20151/Module_eigengenes_07-19-33.csv,2.937406e-19,Oligo,green,8.359966e-16
Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/kME_table_06-44-39.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules/Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151/Module_eigengenes_06-44-39.csv,9.925497e-19,Macrophage,yellow,2.079642e-15
Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/mouse_ACA_20pcntCells_0.4pcn